# Synethetic Data Generation of the Main Sheet

## 1. Intalling Necessary Package

In [2]:
pip install pandas faker


  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/a7/0b/57ac98ca6aa8cd54246b440bc2aa8549e5dcbb99a72a399cab6d66c7f198/Faker-26.0.0-py3-none-any.whl.metadata
Using cached Faker-26.0.0-py3-none-any.whl (1.8 MB)

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Importing Libraries 

In [20]:
import pandas as pd
from faker import Faker
import numpy as np
import random
from datetime import datetime, timedelta

## 3. Generate 5000 Rows of the Main Sheet

In [89]:

# Initialize Faker
fake = Faker()

# Function to generate random date and time within a range
def random_date(start, end):
    return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

# Realistic weather conditions for Colombia
weather_conditions_colombia = [
    "Sunny", "Cloudy", "Rainy", "Partly Cloudy", "Overcast", "Light Rain", "Heavy Rain"
]
weather_probs = [0.2, 0.3, 0.1, 0.1, 0.1, 0.15, 0.05]  # Probabilities for each weather condition

# Types of water commonly used in Colombia for coffee fermentation
types_of_water = ["Spring Water", "Well Water", "Filtered Water"]
water_probs = [0.8, 0.1, 0.1]  # Probabilities for each type of water

# Additives for fermentation
additives = ["None", "Molasses", "Yeast", "Honey", "Sugar"]
additives_probs = [0.8, 0.05, 0.05, 0.05, 0.05]  # Probabilities for each additive

# Coffee varieties
coffee_varieties = ["Special", "Normal"]
coffee_variety_probs = [0.8, 0.2]  # Mostly "Special"

# Fermentation methods
fermentation_methods = ["Washed", "Honey"]
fermentation_method_probs = [0.5, 0.5]  # Equal distribution

# Create a DataFrame with 5000 rows of fake data
data = []
num_records = 5000

# Generate unique Batch IDs and Batch Names
batch_ids = [f"Batch {i+1}" for i in range(num_records)]
batch_names = [f"Batch {i+1}" for i in range(num_records)]

# Define the date range for the last three years
end_date = datetime.now()
start_date = end_date - timedelta(days=3*365)

# Generate realistic distributions for data
for i in range(num_records):
    fermentation_method = np.random.choice(fermentation_methods, p=fermentation_method_probs)
    fermentation_start = fake.date_time_between(start_date=start_date, end_date=end_date)
    fermentation_end = fermentation_start + timedelta(days=random.randint(3, 5), hours=random.randint(0, 24))
    drying_start = fermentation_end + timedelta(hours=random.randint(1, 12))
    drying_end = drying_start + timedelta(days=random.randint(3, 5), hours=random.randint(0, 24))
    
    # Temperature (normal distribution)
    fermentation_temp = np.random.normal(23, 3)  # Normal distribution with mean 23 and std dev 3
    fermentation_temp = np.clip(fermentation_temp, 18.0, 28.0)  # Ensure values are within the range
    
    drying_temp = np.random.normal(23, 3)
    drying_temp = np.clip(drying_temp, 18.0, 28.0)
    
    # Humidity (normal distribution correlated with temperature)
    fermentation_humidity = np.random.normal(75, 10) + (fermentation_temp - 23) * 0.5  # Base 75% with temperature correlation
    fermentation_humidity = np.clip(fermentation_humidity, 60, 90)
    
    drying_humidity = np.random.normal(75, 10) + (drying_temp - 23) * 0.5
    drying_humidity = np.clip(drying_humidity, 60, 90)
    
    # pH Levels (normal distribution)
    ph_level = np.random.normal(4.5, 0.25)
    ph_level = np.clip(ph_level, 4.0, 5.0)
    
    # Fermentation Duration
    fermentation_duration_hours = (fermentation_end - fermentation_start).total_seconds() / 3600  # Duration in hours
    fermentation_duration_days = fermentation_duration_hours / 24  # Duration in days
    
    # SCA Score (separate calculations for Washed and Honey)
    base_sca_score = 85  # Base score
    
    if fermentation_method == 'Washed':
        temp_factor = (fermentation_temp - 23) * 0.2  # Moderate Temperature impact factor
        time_factor = (fermentation_duration_hours - 96) * 0.03  # Moderate Time impact factor
        ph_factor = (ph_level - 4.5) * 1  # Moderate pH level impact factor
        interaction_term = 0.005 * (fermentation_temp - 23) * (fermentation_duration_hours - 96)  # Interaction term
        humidity_factor = (fermentation_humidity - 75) * 0.09  # Moderate Humidity factor
        drying_temp_factor = (drying_temp - 23) * 0.1  # Moderate Drying Temp factor
        random_factor = np.random.normal(0, 0.5)  # Controlled random noise
        non_linear_term = np.log(fermentation_duration_hours) * 0.05  # Non-linear term
        sca_score = base_sca_score + temp_factor + time_factor + ph_factor + interaction_term + humidity_factor + drying_temp_factor + random_factor + non_linear_term
        sca_score = np.clip(sca_score, 80, 90)
    
    elif fermentation_method == 'Honey':
        temp_factor = (fermentation_temp - 23) * 0.35  # Reduced Temperature impact factor
        time_factor = (fermentation_duration_hours - 96) * 0.08  # Reduced Time impact factor
        ph_factor = (ph_level - 4.5) * 2  # Reduced pH level impact factor
        interaction_term = 0.01 * (fermentation_temp - 23) * (fermentation_duration_hours - 96)  # Interaction term
        humidity_factor = (fermentation_humidity - 75) * 0.09  # Reduced Humidity factor
        drying_temp_factor = (drying_temp - 23) * 0.15  # Reduced Drying Temp factor
        barrel_open_hours = random.randint(0, 3)  # Hours the barrel was open, each 3-4 hours within the range
        barrel_open_factor = barrel_open_hours * 0.5  # Significant factor for barrel open hours
        pre_fermentation_factor = pre_fermentation_details * 0.1 if pre_fermentation == "Yes" else 0  # Factor for pre-fermentation
        random_factor = np.random.normal(0, 1.0)  # Controlled noise for more variability
        sca_score = base_sca_score + temp_factor + time_factor + ph_factor + interaction_term + humidity_factor + drying_temp_factor + barrel_open_factor + pre_fermentation_factor + random_factor
        sca_score = np.clip(sca_score, 80, 90)
    
    # Brix Levels (beta distribution scaled to desired range)
    brix_levels = np.random.beta(2, 2) * 5 + 15  # Beta distribution scaled to range 15-20
    
    # Weather Conditions
    num_weather_conditions = random.randint(3, 5)
    fermentation_weather = ",".join(np.random.choice(weather_conditions_colombia, num_weather_conditions, p=weather_probs))
    
    coffee_variety = np.random.choice(coffee_varieties, p=coffee_variety_probs)
    pre_fermentation = np.random.choice(["Yes", "No"], p=[0.3, 0.7])  # More likely to be "No"
    pre_fermentation_details = random.randint(0, 24) if pre_fermentation == "Yes" else 0
    weather_conditions = np.random.choice(weather_conditions_colombia, p=weather_probs)
    type_of_water = np.random.choice(types_of_water, p=water_probs)
    additives_fermentation = np.random.choice(additives, p=additives_probs)
    
    # Record for barrels (only for Honey)
    barrel_open_hours = barrel_close_hours = ''
    if fermentation_method == 'Honey':
        barrel_open_hours = random.randint(0, 3)  # Hours the barrel was open, each 3-4 hours within the range
        barrel_close_hours = random.randint(60, 120) - barrel_open_hours  # Hours the barrel was closed
    
    # Fermentation records
    num_records_fermentation = random.randint(3, 5)
    fermentation_records_dates = [random_date(fermentation_start, fermentation_end) for _ in range(num_records_fermentation)]
    fermentation_records = ",".join([f"ferm_{batch_ids[i]}_{dt.isoformat()}" for dt in fermentation_records_dates])
    
    # Drying records
    num_records_drying = random.randint(3, 5)
    drying_records_dates = [random_date(drying_start, drying_end) for _ in range(num_records_drying)]
    drying_records = ",".join([f"dry_{batch_ids[i]}_{dt.isoformat()}" for dt in drying_records_dates])
    
    # Form submission dates
    form_submitted = random_date(fermentation_start, fermentation_start + timedelta(hours=12))
    form_modified = random_date(fermentation_end, fermentation_end + timedelta(hours=12))
    
    record = [
        batch_ids[i], batch_names[i], coffee_variety, 
        fermentation_end.strftime("%m/%d/%Y"), 
        fermentation_method, type_of_water if fermentation_method == "Washed" else '', 
        additives_fermentation, int(sca_score),
        fermentation_records,
        round(fermentation_temp, 1), 
        round(ph_level, 3), 
        round(brix_levels, 1), 
        int(fermentation_humidity),
        fermentation_weather, 
        fermentation_start.strftime("%m/%d/%Y %I:%M %p"), 
        fermentation_end.strftime("%m/%d/%Y %I:%M %p"),
        drying_records,
        drying_start.strftime("%m/%d/%Y %I:%M %p"), 
        drying_end.strftime("%m/%d/%Y %I:%M %p"), 
        form_submitted.strftime("%m/%d/%Y %I:%M %p"), form_modified.strftime("%m/%d/%Y %I:%M %p"), 
        round(drying_temp, 1), 
        round(drying_humidity, 3), 
        barrel_open_hours if fermentation_method == 'Honey' else '', 
        barrel_close_hours if fermentation_method == 'Honey' else '',
        pre_fermentation if fermentation_method == 'Honey' else '',
        pre_fermentation_details if fermentation_method == 'Honey' else ''
    ]

    data.append(record)

# Define column names
columns = [
    "Batch ID", "Batch Name", "Coffee Variety", "Harvest Date", "Fermentation Method", 
    "Type of Water Used", "Additives for fermentation", "SCA Score", "Fermentation Records", 
    "Average Temp", "Average PH", "Average Brix", "Average Humidity", "Fermentation Weather", 
    "Fermentation Start", "Fermentation End", "Drying Records", "Drying Start Date/Time", 
    "Drying End Date/Time", "Form Submitted Date/Time", "Form Modified Date/Time", 
    "Average Drying Temp", "Average Drying Humidity", "Barrel Open in (Hour)", "Barrel Close in (Hour)",
    "Pre-fermentation for Honey", "Pre-fermentation Details (Hour) for Honey"
]

# Create DataFrame
df = pd.DataFrame(data, columns=columns)

# Please replace the file path to fit your own work path
# Specify the file path
file_path = '/Users/yuyichen/Desktop/Summer 2024/Community Project/fake_main_data.csv'

# Save to CSV
df.to_csv(file_path, index=False)

print(f"File saved to {file_path}")


File saved to /Users/yuyichen/Desktop/Summer 2024/Community Project/fake_data_updated.csv
